In [5]:
import requests
import time
import random
import sqlite3
from bs4 import BeautifulSoup
import re
import cloudscraper
from requests_html import HTMLSession

conn = sqlite3.connect('nulled.db')
cursor = conn.cursor()

Session = HTMLSession()


user_agents = [ 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36', 
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36', 
    'Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148', 
    'Mozilla/5.0 (Linux; Android 11; SM-G960U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Mobile Safari/537.36',
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
]

In [6]:
def create_tables(c):
    cursor = c.cursor()
    # Create a table if it doesn't exist
    cursor.execute('''CREATE TABLE IF NOT EXISTS urls (url TEXT)''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS topics (url TEXT,title text, user_id text,author text,timestamp text)''')
    c.commit()
create_tables(conn) 

In [7]:
urls = ["https://www.nulled.to/#!Home","https://www.nulled.to/#!Pentesting","https://www.nulled.to/#!Leaks", "https://www.nulled.to/#!Coding" , "https://www.nulled.to/#!Monetizing", "https://www.nulled.to/#!Marketplace",  ]

In [8]:
def scrape_forums(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    all_links = soup.find_all("a")
    pattern = re.compile(r".*/forum/.*")

    # Filter and print the links that match the pattern
    for link in all_links:
        href = link.get("href")
        if href and pattern.match(href):
            cursor.execute('''INSERT INTO urls (url) VALUES (?)''', (href,))
            conn.commit()


In [9]:
def scrape_topics(soup):
    #print(soup)
    topics = soup.find_all("tr",class_="__topic")
    for topic in topics:
        try:
            url = topic.find("td",class_="col_f_content").find("a").get("href")
            #print(url)
            title = topic.find("td",class_="col_f_content").find("a").text.strip().split("\n")[0]
            #print(title)
            author = topic.find("td",class_="col_f_content").find("a", class_="url").text.strip()
            #print(author)
            user_id = topic.find("td",class_="col_f_content").find("a", class_="url").get("href").split("/")[-1]
            #print(user_id)
            timestamp = topic.find("td",class_="col_f_content").find("a",class_="topic_title highlight_unread").get("title").split("started")[-1].replace("-","")
            #if timestamp contains word today, replace it with today's date with the following format 28 December 2022
            if "Today" in timestamp:
                timestamp = time.strftime("%d %B %Y")

            if "Yesterday" in timestamp:
                #add yesterday to timestamp
                timestamp = time.strftime("%d-%B-%Y")

            #print(timestamp)
            cursor.execute('''INSERT or ignore INTO topics (url,title,user_id,author,timestamp) VALUES (?,?,?,?,?)''', (url,title,user_id,author,timestamp))
            conn.commit()
            print(f"inserted {url}")
        except Exception as e:
            print(e)

def scrape_forum_pages(url):
    user_agent = random.choice(user_agents) 
    headers = {'User-Agent': user_agent}
    
    response = Session.get(url,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    scrape_topics(soup)
    #print(soup)
    try:
        page_count = soup.find("li", class_="pagejump").text.strip().split(" ")[-1]
        print(page_count)
    except:
        print("Possible cloudflare")


In [10]:
## update the select statement so it only gets the once that have not been scraped before.
cursor.execute("select url from urls ")
rows = cursor.fetchall()
for row in rows:
    scrape_forum_pages(row[0])
    time.sleep(5)

inserted https://www.nulled.to/topic/939342-refund-god-8-years-experience-20000-limits-usa-canada-worldwide-fast-responses/
inserted https://www.nulled.to/topic/1494871-amazon-god-1-usacanada-macan-refunds-watch-stores-paypal-instant-walmart-hot/
inserted https://www.nulled.to/topic/828843-all-amazon-domains-instant-worldwide-5-fee-starting-best-german-amazon-refunder-fastest-responding-time-lowest-fees/
inserted https://www.nulled.to/topic/1397970-worldwide-refunds/
inserted https://www.nulled.to/topic/1389272-amazon-insider-worldwide-top-vouched-nulled-us-canada-europe-uk-worldwide-mentorship-clothing-electronics/
inserted https://www.nulled.to/topic/1296005-samsung-usa-god-10-fee-usa-canada-highly-vouched-amazon-mentorship-kohls-instant-instant-stores-999-success-rate/
inserted https://www.nulled.to/topic/1544153-⭐amazon-paypal-ua-zalando-highest-success-rate-fast-and-reliable-redarmyref/
inserted https://www.nulled.to/topic/1397605-back-to-school-sale-lowest-fees-on-nulled-1-usa-ca

KeyboardInterrupt: 